In [ ]:
import json
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.patches import Patch
import pandas as pd
import numpy as np

from app.data.utils.bigquery import BigQuery

bq = BigQuery()

In [ ]:
sql = '''
SELECT campaign_id, line_item_id, COUNT(DISTINCT creative_id) AS n_creatives, MIN(ds) AS min_ds, MAX(ds) AS max_ds
FROM `meli-bi-data.SBOX_DSPCREATIVOS.BQ_PRINTS_CLICKS_PER_HOUR`
GROUP BY 1,2
HAVING n_creatives > 1
'''

df = bq.run_query(sql)
df

In [ ]:
campaign_id = 5784
line_item_id = 12654
sql = f'''
WITH grouped AS
(
    SELECT ds, campaign_id, line_item_id, creative_id, SUM(n_clicks) AS n_clicks, SUM(n_prints) AS n_prints
    FROM `meli-bi-data.SBOX_DSPCREATIVOS.BQ_PRINTS_CLICKS_PER_HOUR`
    WHERE campaign_id = {campaign_id} AND line_item_id = {line_item_id}
    GROUP BY 1,2,3,4
)

SELECT *, n_clicks / n_prints AS ctr
FROM grouped
ORDER BY ds, creative_id
'''

df = bq.run_query(sql)

In [ ]:
pivot = pd.pivot_table(data=df, index=['ds'], columns=['creative_id'], values='n_prints')
pivot_ctr = pd.pivot_table(data=df, index=['ds'], columns=['creative_id'], values='ctr')

In [ ]:
std_pivot = (pivot.T / pivot.agg(sum, axis = 1)).T

In [ ]:
df_grouped = df.groupby('creative_id')[['n_prints', 'n_clicks']].sum()

In [ ]:
df_grouped = pd.concat([df_grouped, pd.DataFrame(df_grouped.sum()).T.rename({0: 'Total'})])

In [ ]:
df_grouped['ctr'] = df_grouped['n_clicks'] / df_grouped['n_prints']
df_grouped

In [ ]:
sns.set_theme()
fig, (ax1, ax2) = plt.subplots(2, figsize=(7, 9))

fig.suptitle(f'campaign_id = {df["campaign_id"].values[0]}, line_item_id = {df["line_item_id"].values[0]}')

std_pivot.plot.bar(stacked=True, ax = ax2)
ax2.set_ylabel('Prints')
ax2.get_legend().remove()

pivot_ctr.set_index(ax2.get_xticks(), inplace=True)
pivot_ctr.plot(ax=ax1)
ax1.set_ylabel('ctr')
ax1.set_xlim(ax2.get_xlim())
ax1.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax1.set_xlabel('')

plt.tight_layout()
plt.savefig(f'bidder_example_{df["campaign_id"].values[0]}_{df["line_item_id"].values[0]}.png', dpi=300)